In [ ]:
import os
import dill
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import (matthews_corrcoef,auc,roc_auc_score, average_precision_score, accuracy_score,
                             precision_score, recall_score, f1_score, brier_score_loss,
                             roc_curve, precision_recall_curve)
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

import shap
import matplotlib.pyplot as plt

In [ ]:
# Dictionary 
D_ICD_DIAGNOSES = pd.read_csv('/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz')
D_ICD_DIAGNOSES.columns = D_ICD_DIAGNOSES.columns.str.upper()
D_ICD_DIAGNOSES['ICD_VERSION'] = 9
D_ICD_DIAGNOSES = D_ICD_DIAGNOSES[['ICD9_CODE','ICD_VERSION','LONG_TITLE']]
D_ICD_DIAGNOSES.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']
print(D_ICD_DIAGNOSES.shape)

D_4_DIAGNOSES = pd.read_csv('C:/MIMIC IV/3.1/hosp/D_ICD_DIAGNOSES.csv.gz')
D_4_DIAGNOSES.columns = D_4_DIAGNOSES.columns.str.upper()
D_4_DIAGNOSES.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']
print(D_4_DIAGNOSES.shape)

D_ICD_DIAGNOSES = pd.concat([D_ICD_DIAGNOSES,D_4_DIAGNOSES])
D_ICD_DIAGNOSES = D_ICD_DIAGNOSES.drop_duplicates(keep='first')
print(D_ICD_DIAGNOSES.shape)

d_iii_icd_procedures = pd.read_csv('/mimic-iii-clinical-database-1.4/d_icd_procedures.csv.gz')
d_iii_icd_procedures = d_iii_icd_procedures.rename(columns={'ICD9_CODE':'ICD_CODE'})
d_iii_icd_procedures = d_iii_icd_procedures.rename(columns={'LONG_TITLE':'ICD_TEXT'})
d_iii_icd_procedures['ICD_VERSION'] = 9
d_iii_icd_procedures = d_iii_icd_procedures[['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']]
d_iii_icd_procedures['ICD_CODE'] = d_iii_icd_procedures['ICD_CODE'].astype(str)

d_iv_icd_procedures = pd.read_csv('C:/MIMIC IV/3.1/hosp/d_icd_procedures.csv.gz')
d_iv_icd_procedures.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']
d_iv_icd_procedures['ICD_CODE'] = d_iv_icd_procedures['ICD_CODE'].astype(str)

P_ICD_procedures = pd.concat([d_iii_icd_procedures,d_iv_icd_procedures])
P_ICD_procedures = P_ICD_procedures.drop_duplicates(keep='first')

dd = P_ICD_procedures[P_ICD_procedures.duplicated(subset=['ICD_TEXT'],keep=False)].sort_values(by='ICD_TEXT')
dd = (
    dd.groupby('ICD_TEXT')
    .agg(
        final_ICD_CODE=('ICD_CODE', lambda x: max(x, key=len)), 
        all_ICD_CODEs=('ICD_CODE', lambda x: list(x))            
    )
    .reset_index()
)

P_ICD_procedures = (
    P_ICD_procedures.assign(code_len=P_ICD_procedures['ICD_CODE'].str.len())
    .sort_values('code_len', ascending=False)
    .drop_duplicates(subset=['ICD_TEXT'], keep='first')
    .drop(columns='code_len')
)

print(P_ICD_procedures.ICD_VERSION.value_counts())

In [ ]:
def sclar_coder(rundf,num_v,cate_v):
    scaler = MinMaxScaler()
    rundf[num_v] = scaler.fit_transform(rundf[num_v])
    
    label_encoder = LabelEncoder()
    for col in cate_v:
        rundf[col] = label_encoder.fit_transform(rundf[col])

    return rundf

def print_top_features(all_importances, D_ICD_DIAGNOSES, P_ICD_procedures, sort_by='xgboost', top_n=20):

    # Print header
    print(f"---------- Top {top_n} Feature Importances (Sorted by {sort_by}) ----------")

    # Sort features by specified model and get top N
    top_features = all_importances.sort_values(by=sort_by, ascending=False).Feature.head(top_n).values

    for feature in top_features:
        if '_' in feature:
            try:
                # Split feature name and use the second part as ICD_CODE
                icd_code = feature.split('_')[-1]
                
                # Check D_ICD_DIAGNOSES
                diag_match = D_ICD_DIAGNOSES[D_ICD_DIAGNOSES['ICD_CODE'] == icd_code]
                if not diag_match.empty:
                    print(f"{feature}: {diag_match['ICD_TEXT'].values[0]}")
                
                # Check P_ICD_procedures
                proc_match = P_ICD_procedures[P_ICD_procedures['ICD_CODE'] == icd_code]
                if not proc_match.empty:
                    print(f"{feature}: {proc_match['ICD_TEXT'].values[0]}")
                
                # If no match found in either DataFrame
                if diag_match.empty and proc_match.empty:
                    print(f"{feature}: No matching ICD code found")
                    
            except IndexError:
                print(f"{feature}: No valid ICD code (malformed feature name)")
        else:
            print(feature)
    return top_features

## ML

In [3]:
class BaseModel:
    @staticmethod
    def get_classifier(model_type='random_forest',label_column=None, params=None):
        """Return a classifier based on model_type with given parameters."""
        if params is None:
            params = {}
        if model_type == 'Random Forest':
            return RandomForestClassifier(**params)
        elif model_type == 'Decision Tree':
            return DecisionTreeClassifier(**params)
        elif model_type == 'Adaboost':
            return AdaBoostClassifier(**params)
        elif model_type == 'XGboost':
            return XGBClassifier(**params)
        elif model_type == 'LR':
            return LogisticRegression(**params)
        elif model_type == 'SVM':
            return SVC(**params)
        elif model_type == 'MLP':
            return MLPClassifier(**params)
        elif model_type == 'ensemble':
            base_models = [
                ('rf', RandomForestClassifier()),
                ('xgb', XGBClassifier(eval_metric='mlogloss'))
            ]
            base_models = [(f'model_{i}', model) for i, model in enumerate(base_models)]
            return VotingClassifier(estimators=base_models, voting='soft', n_jobs=1)
        else:
            raise ValueError(f"Unknown model_type: {model_type}")

    @staticmethod
    def bootstrap(train, valid, test, label_column, models, n=10, confidence=0.95):
        """Perform bootstrap cross-validation, evaluate models"""
        # Validate inputs
        if label_column not in train.columns or label_column not in test.columns:
            raise ValueError(f"Label column '{label_column}' not found in train or test DataFrame.")


        skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)
        features = train.columns.difference([label_column])

        all_scores = []
        all_importances = []
        all_AUROC = []
        all_AUPRC = []

        for model_type, params in models:
            scores_model = []
            importances_model = pd.DataFrame({'Feature': features})
            this_AUROC = []
            this_AUPRC = []

            for fold, (train_index, val_index) in enumerate(tqdm(skf.split(train, train[label_column]), total=n, desc=f'Model: {model_type}', ascii=True)):
                # Use train and validation splits from StratifiedKFold
                X_train = train.iloc[train_index][features]
                y_train = train.iloc[train_index][label_column]
                X_val = valid[features]
                y_val = valid[label_column]
                X_test = pd.concat([test[features], train.iloc[val_index][features]])
                y_test = pd.concat([test[label_column], train.iloc[val_index][label_column]])

                # Initialize and train the model
                est = BaseModel.get_classifier(model_type, label_column, params)
                if model_type == 'autogluon':
                    train_data = train.iloc[train_index][list(features) + [label_column]].copy()
                    est.fit(train_data, hyperparameter_tune_kwargs='auto')
                else:
                    if model_type == 'tabnet':
                        est.fit(
                            X_train.to_numpy(), y_train,
                            eval_set=[(X_val.to_numpy(), y_val)],
                            eval_metric=['auc']
                        )
                    else:
                        est.fit(X_train, y_train)

                # Get predictions and probabilities
                if model_type == 'autogluon':
                    y_pred = est.predict(test[list(features) + [label_column]])
                    y_prob = est.predict_proba(test[list(features) + [label_column]]).iloc[:, 1].values
                elif model_type == 'tabnet':
                    y_pred = est.predict(X_test.to_numpy())
                    y_prob = np.zeros_like(y_pred)
                else:
                    y_pred = est.predict(X_test)
                    try:
                        y_prob = est.predict_proba(X_test)[:, 1]
                    except AttributeError:
                        y_prob = np.zeros_like(y_pred)

                # Evaluate AUROC
                fpr, tpr, _ = roc_curve(y_test, y_prob)
                roc_auc = auc(fpr, tpr)
                this_AUROC.append({'roc_auc': roc_auc, 'fpr': fpr, 'tpr': tpr})

                # Evaluate AUPRC
                precision, recall, _ = precision_recall_curve(y_test, y_prob)
                prc_auc = auc(recall, precision)
                this_AUPRC.append({'prc_auc': prc_auc, 'precision': precision, 'recall': recall})

                # Compute other metrics
                accuracy = accuracy_score(y_test, y_pred)
                precision_score_val = precision_score(y_test, y_pred, average='weighted', zero_division=0)
                recall_score_val = recall_score(y_test, y_pred, average='weighted', zero_division=0)
                f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
                mcc = matthews_corrcoef(y_test, y_pred)
                brier = brier_score_loss(y_test, y_prob) if y_prob is not None else np.nan

                # Store evaluation metrics
                scores_model.append({
                    'ROC_AUC': roc_auc,
                    'PRC_AUC': prc_auc,
                    'Accuracy': accuracy,
                    'Precision': precision_score_val,
                    'Recall': recall_score_val,
                    'F1': f1,
                    'MCC': mcc,
                    'Brier': brier
                })

                # Calculate feature importance
                if model_type == 'autogluon':
                    importance = est.feature_importance(X_test).importance.values
                elif hasattr(est, 'feature_importances_'):
                    importance = est.feature_importances_
                elif hasattr(est, 'coef_') and est.coef_.ndim == 1:
                    importance = np.abs(est.coef_)
                else:
                    importance = np.zeros(len(features))

                importances_model[f'fold_{fold}'] = importance

            # Aggregate scores
            scores_model = pd.DataFrame(scores_model)
            all_scores.append(scores_model)

            # Aggregate feature importance
            importances_model[f'{model_type}_mean'] = importances_model.filter(like='fold_').mean(axis=1)
            all_importances.append(importances_model[['Feature', f'{model_type}_mean']])

            # Calculate confidence intervals for AUROC and AUPRC
            lower_bound = (1 - confidence) / 2
            upper_bound = 1 - lower_bound

            auroc_vals = [x['roc_auc'] for x in this_AUROC]
            auprc_vals = [x['prc_auc'] for x in this_AUPRC]
            auroc_ci = (np.percentile(auroc_vals, lower_bound * 100), np.percentile(auroc_vals, upper_bound * 100))
            auprc_ci = (np.percentile(auprc_vals, lower_bound * 100), np.percentile(auprc_vals, upper_bound * 100))

            all_AUROC.append({
                'model': model_type,
                'mean': np.mean(auroc_vals),
                'ci_lower': auroc_ci[0],
                'ci_upper': auroc_ci[1],
                'curves': this_AUROC
            })
            all_AUPRC.append({
                'model': model_type,
                'mean': np.mean(auprc_vals),
                'ci_lower': auprc_ci[0],
                'ci_upper': auprc_ci[1],
                'curves': this_AUPRC
            })


        # Combine all scores
        all_scores_df = pd.concat([df.assign(Model=model_type) for (model_type, _), df in zip(models, all_scores)], ignore_index=True)
        all_scores_summary = all_scores_df.groupby('Model').agg(
            {col: lambda x: f"{np.mean(x):.4f} ({np.percentile(x, lower_bound * 100):.4f}-{np.percentile(x, upper_bound * 100):.4f})"
             for col in all_scores_df.columns if col != 'Model'}
        ).reset_index()

        # Combine all importances
        all_importances_df = pd.concat([df.rename(columns={f'{model_type}_mean': model_type}) for (model_type, _), df in zip(models, all_importances)], axis=1)
        all_importances_df = all_importances_df.loc[:, ~all_importances_df.columns.duplicated()]

        return all_AUROC, all_AUPRC, all_scores_summary, all_importances_df

In [ ]:
III_IV = pd.read_csv('/IV_III.csv')
III_IV = III_IV[III_IV['Group_Va_uti']==1]
print(III_IV.shape)

basic = ['GENDER', 'ADMISSION_TYPE', 'FIRST_CAREUNIT', 'AGE']
groups = ['Group_AKI', 'Group_CKD', 'Group_PCOS', 'Group_Neoplasm_ovary', 'Group_Endometriosis', 'Group_Leiomyoma']

label = ['LOS_Hospital','DIEINHOSPITAL','Readmission_30','Multiple_ICUs','sepsis_all', 'FirstICU24_AKI_ALL','ICU_within_12hr_of_admit']

print('Diag:',len(Diag), 'full_Diag:', len(full_Diag),'Proc:',len(Proc),'full_Proc:',len(full_Proc),'Med:',len(Med),'TS:',len(TS),'label:',len(label))


num_v = Med + TS + ['AGE']

cate_v = ['GENDER','ADMISSION_TYPE','FIRST_CAREUNIT'] + Diag + Proc + label


III_IV[Diag + Proc + Med] = III_IV[Diag + Proc + Med].fillna(0)

III_IV[TS] = III_IV[TS].fillna(III_IV[TS].median())

def sclar_coder(rundf,num_v,cate_v):
    scaler = MinMaxScaler()
    rundf[num_v] = scaler.fit_transform(rundf[num_v])
    
    label_encoder = LabelEncoder()
    for col in cate_v:
        rundf[col] = label_encoder.fit_transform(rundf[col])

    return rundf

rundf = sclar_coder(III_IV,num_v,cate_v)
rundf.shape

In [ ]:
def runbase(df,thislabel,ft,topf):
    ft = ft + [thislabel]
    print(thislabel,Counter(df[thislabel]),'len(ft):',len(ft))

    Train, Test = train_test_split(df, test_size=0.3, random_state=42)
    Valid, Test = train_test_split(Test, test_size=0.25, random_state=42)
    print(Train.shape,Valid.shape,Test.shape)
    print('Train:',Counter(Train[thislabel]))
    print('Test:',Counter(Test[thislabel]))

    all_AUROC, all_AUPRC, all_scores, all_importances = BaseModel.bootstrap(
       Train[ft], Valid[ft], Test[ft], thislabel, models, n=1000, confidence=0.95
    )

    all_importances['XGB_RF'] = all_importances['XGboost'] + all_importances['Random Forest'] 
    top_xgboost = print_top_features(all_importances, D_ICD_DIAGNOSES, P_ICD_procedures, sort_by='XGboost', top_n=topf)
    top_rf = print_top_features(all_importances, D_ICD_DIAGNOSES, P_ICD_procedures, sort_by='Random Forest', top_n=topf)
    top_XGB_RF = print_top_features(all_importances, D_ICD_DIAGNOSES, P_ICD_procedures, sort_by='XGB_RF', top_n=topf)
    
    print('label',thislabel)
    print('top_xgboost',list(top_xgboost))
    print('top_rf',list(top_rf))
    print('top_XGB_RF',list(top_XGB_RF))
    
    return all_AUROC, all_AUPRC, all_scores, all_importances

In [ ]:
ft = basic + Diag + Proc + Med + TS
print(len(ft))

models = [
    ('SVM', {'probability': True}),
    ('LR', {}),
    ('Decision Tree',{}),
    ('Random Forest', {}),
    ('Adaboost',{}),
    ('XGboost', {}),
         ]

In [ ]:
print(label)

In [ ]:
t_label = 'DIEINHOSPITAL'
t_f = 'ft'
all_AUROC, all_AUPRC, all_scores, all_importances = runbase(rundf[rundf.MIMIC == mic],t_label,ft,5)
all_scores.style.highlight_max(color='lightgreen', axis=0)

In [ ]:
t_label = 'Readmission_30'
t_f = 'ft'
all_AUROC, all_AUPRC, all_scores, all_importances = runbase(rundf[rundf.MIMIC == mic],'Readmission_30',ft,20)
all_scores.style.highlight_max(color='lightgreen', axis=0)

In [ ]:
t_label = 'Multiple_ICUs'
t_f = 'ft'
all_AUROC, all_AUPRC, all_scores, all_importances = runbase(rundf[rundf.MIMIC == mic],'Multiple_ICUs',ft,5)
all_scores.style.highlight_max(color='lightgreen', axis=0)

In [ ]:
t_label = 'sepsis_all'
t_f = 'ft'
all_AUROC, all_AUPRC, all_scores, all_importances = runbase(rundf[rundf.MIMIC == mic],'sepsis_all',ft,5)
all_scores.style.highlight_max(color='lightgreen', axis=0)

In [ ]:
t_label = 'FirstICU24_AKI_ALL'
t_f = 'ft'
all_AUROC, all_AUPRC, all_scores, all_importances = runbase(rundf[rundf.MIMIC == mic],'FirstICU24_AKI_ALL',ft,5)
all_scores.style.highlight_max(color='lightgreen', axis=0)

In [ ]:
t_label = 'LOS_Hospital'
t_f = 'ft'
all_AUROC, all_AUPRC, all_scores, all_importances = runbase(rundf[rundf.MIMIC == mic],'LOS_Hospital',ft,5)
all_scores.style.highlight_max(color='lightgreen', axis=0)

In [ ]:
t_label = 'ICU_within_12hr_of_admit'
t_f = 'ft'
ft_icu12 = basic + Diag
print(len(ft_icu12))
all_AUROC, all_AUPRC, all_scores, all_importances = runbase(rundf[rundf.MIMIC == mic],'ICU_within_12hr_of_admit',ft_icu12,15)
all_scores.style.highlight_max(color='lightgreen', axis=0)